In [4]:
import chess
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder

# Paths to saved models and scalers
piece_model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
time_model_path = r"C:\Users\shaik\Desktop\time_v2.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
scaler_X_path = r"C:\Users\shaik\Desktop\scaler_X.pkl"
scaler_y_path = r"C:\Users\shaik\Desktop\scaler_y.pkl"

# Load the trained Piece Position and Time models
piece_model = xgb.Booster()
piece_model.load_model(piece_model_path)

time_model = xgb.Booster()
time_model.load_model(time_model_path)

# Load the move label encoder
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)

# Load the scalers
with open(scaler_X_path, 'rb') as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, 'rb') as f:
    scaler_y = pickle.load(f)

print("Both Piece Position and Time models, along with scalers, loaded successfully.")

# Initialize the chess board
board = chess.Board()

# Define the function to extract combined features for both models
def extract_combined_features(board, player_id, game_phase):
    # Extract board features for piece position prediction
    piece_mapping = {
        'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
        'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6,
        '.': 0  # Empty squares
    }
    board_features = [piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES]
    
    # Combine features for the time model (adjust as needed for your game data)
    game_mode_encoded = 1  # Example: 1 for Rapid; adjust according to your selected game mode
    white_time_left_processed = 180  # Example time, adjust based on game state
    black_time_left_processed = 180
    total_seconds = 180  # Example value
    piece_count = 32 - len(board.move_stack)
    
    time_model_features = np.array([
        game_mode_encoded, white_time_left_processed, black_time_left_processed,
        total_seconds, game_phase, piece_count
    ])
    
    # Scale time model features
    scaled_time_features = scaler_X.transform(time_model_features.reshape(1, -1))

    return np.array(board_features), scaled_time_features

# Main game loop
while not board.is_game_over():
    # Extract game phase based on move count
    move_count = len(board.move_stack)
    game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)

    if board.turn == chess.WHITE:
        # Example user move input (for testing purposes, assume move from legal moves)
        print("Your move. Legal moves:", list(board.legal_moves))
        user_move = input("Enter your move in SAN: ")
        board.push_san(user_move)
    else:
        # AI's move and time prediction
        player_id = 0  # Example player ID, adjust if using specific player data
        
        board_features, scaled_time_features = extract_combined_features(board, player_id, game_phase)
        
        # Predict the move
        piece_dmatrix = xgb.DMatrix(np.array([board_features]))
        predicted_move_encoded = int(piece_model.predict(piece_dmatrix)[0])
        predicted_move = move_encoder.inverse_transform([predicted_move_encoded])[0]
        ai_move = chess.Move.from_uci(predicted_move)
        
        # Predict the time taken for the move
        time_dmatrix = xgb.DMatrix(scaled_time_features)
        predicted_time_scaled = time_model.predict(time_dmatrix)[0]
        predicted_time = scaler_y.inverse_transform([[predicted_time_scaled]])[0][0]
        
        # Make and print the move
        if ai_move in board.legal_moves:
            board.push(ai_move)
            print(f"AI move: {ai_move} (Predicted time taken: {np.expm1(predicted_time):.2f} seconds)")  # Reverse log transform
        else:
            print("Predicted move is illegal, retrying...")

# End of game
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())


Both Piece Position and Time models, along with scalers, loaded successfully.
Your move. Legal moves: [Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('b1a3'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('a2a3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e2e4'), Move.from_uci('d2d4'), Move.from_uci('c2c4'), Move.from_uci('b2b4'), Move.from_uci('a2a4')]
AI move: d7d5 (Predicted time taken: 7.08 seconds)
Your move. Legal moves: [Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('a1a2'), Move.from_uci('a3a4'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e

KeyboardInterrupt: 

In [9]:
import chess
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder

# Paths to saved models and scalers
piece_model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
time_model_path = r"C:\Users\shaik\Desktop\time_v2.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
scaler_X_path = r"C:\Users\shaik\Desktop\scaler_X.pkl"
scaler_y_path = r"C:\Users\shaik\Desktop\scaler_y.pkl"

# Load the trained Piece Position and Time models
piece_model = xgb.Booster()
piece_model.load_model(piece_model_path)

time_model = xgb.Booster()
time_model.load_model(time_model_path)

# Load the move label encoder
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)

# Load the scalers
with open(scaler_X_path, 'rb') as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, 'rb') as f:
    scaler_y = pickle.load(f)

print("Both Piece Position and Time models, along with scalers, loaded successfully.")

# Initialize the chess board
board = chess.Board()

# Define the function to extract combined features for both models
def extract_combined_features(board, player_id, game_phase):
    # Extract board features for piece position prediction
    piece_mapping = {
        'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
        'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6,
        '.': 0  # Empty squares
    }
    board_features = [piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES]
    
    # Combine features for the time model (adjust as needed for your game data)
    game_mode_encoded = 1  # Example: 1 for Rapid; adjust according to your selected game mode
    white_time_left_processed = 180  # Example time, adjust based on game state
    black_time_left_processed = 180
    total_seconds = 180  # Example value
    piece_count = 32 - len(board.move_stack)
    
    time_model_features = np.array([
        game_mode_encoded, white_time_left_processed, black_time_left_processed,
        total_seconds, game_phase, piece_count
    ])
    
    # Scale time model features
    scaled_time_features = scaler_X.transform(time_model_features.reshape(1, -1))

    return np.array(board_features), scaled_time_features



# Main game loop
while not board.is_game_over():
    # Display the current board position
    print("\nCurrent Board Position:")
    print(board)

    # Extract game phase based on move count
    move_count = len(board.move_stack)
    game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)

    if board.turn == chess.WHITE:
        # Example user move input (for testing purposes, assume move from legal moves)
        print("Your move. Legal moves:", list(board.legal_moves))
        user_move = input("Enter your move in SAN: ")
        try:
            board.push_san(user_move)
        except ValueError:
            print("Illegal move. Please try again.")
            continue  # Skip to the next loop iteration if the move is illegal
    else:
        # AI's move and time prediction
        player_id = 0  # Example player ID, adjust if using specific player data
        
        board_features, scaled_time_features = extract_combined_features(board, player_id, game_phase)
        
        # Predict the move
        piece_dmatrix = xgb.DMatrix(np.array([board_features]))
        predicted_move_encoded = int(piece_model.predict(piece_dmatrix)[0])
        predicted_move = move_encoder.inverse_transform([predicted_move_encoded])[0]
        ai_move = chess.Move.from_uci(predicted_move)
        
        # Attempt up to 5 alternative moves if illegal
        retries = 5
        while retries > 0 and ai_move not in board.legal_moves:
            print("Predicted move is illegal, trying next best move...")
            retries -= 1
            piece_dmatrix = xgb.DMatrix(np.array([board_features]))  # Retry with the same features
            predicted_move_encoded = int(piece_model.predict(piece_dmatrix)[0])
            predicted_move = move_encoder.inverse_transform([predicted_move_encoded])[0]
            ai_move = chess.Move.from_uci(predicted_move)

        if ai_move in board.legal_moves:
            # Predict the time taken for the move
            time_dmatrix = xgb.DMatrix(scaled_time_features)
            predicted_time_scaled = time_model.predict(time_dmatrix)[0]
            predicted_time = scaler_y.inverse_transform([[predicted_time_scaled]])[0][0]
            
            # Make the move
            board.push(ai_move)
            print(f"AI move: {ai_move} (Predicted time taken: {np.expm1(predicted_time):.2f} seconds)")  # Reverse log transform
        else:
            print("AI could not find a legal move, ending game.")
            break

# End of game
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())


Both Piece Position and Time models, along with scalers, loaded successfully.

Current Board Position:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Your move. Legal moves: [Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('b1a3'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('a2a3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e2e4'), Move.from_uci('d2d4'), Move.from_uci('c2c4'), Move.from_uci('b2b4'), Move.from_uci('a2a4')]

Current Board Position:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
P . . . . . . .
. P P P P P P P
R N B Q K B N R
AI move: d7d5 (Predicted time taken: 7.08 seconds)

Current Board Position:
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p

In [ ]:
import chess
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder

# Load models, encoders, and stockfish engine
model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
stockfish_path = r"C:\Users\shaik\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
bst = xgb.Booster()
bst.load_model(model_path)
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Initialize board and player encoder
board = chess.Board()
data_path = r"C:\Users\shaik\Desktop\updated_time_control_rest_1.csv"
df = pd.read_csv(data_path)
top_50_players = pd.concat([df['White'], df['Black']]).value_counts().head(50).index.tolist()
player_encoder = LabelEncoder()
player_encoder.fit([player.lower() for player in pd.concat([df['White'], df['Black']]).unique()])
selected_player = top_50_players[0]
selected_player_id = player_encoder.transform([selected_player.lower()])[0]

def board_to_features(board):
    piece_mapping = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6, 'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6, '.': 0}
    return np.array([piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES])

def extract_features(board, player_id, game_phase):
    return np.hstack([board_to_features(board), game_phase, player_id])

def select_best_move(board, player_id, game_phase):
    # Get top moves from Stockfish
    result = engine.analyse(board, chess.engine.Limit(depth=10), multipv=10)
    stockfish_moves_uci = [info['pv'][0] for info in result if info['pv'][0] in board.legal_moves]
    
    print("Stockfish Top Moves (UCI):", stockfish_moves_uci)

    # Extract features for model evaluation
    features_list = []
    for move_uci in stockfish_moves_uci:
        board.push(move_uci)
        features_list.append(extract_features(board, player_id, game_phase))
        board.pop()

    # Model predicts the best move based on highest score
    dmatrix = xgb.DMatrix(np.array(features_list))
    predictions = bst.predict(dmatrix)
    best_move_index = np.argmax(predictions)
    best_move_uci = stockfish_moves_uci[best_move_index]
    
    # Apply the best move directly, assuming legality from Stockfish's output
    board.push(best_move_uci)
    print(f"AI selected move: {best_move_uci} (based on model ranking)")

    # Display updated board
    print("\nUpdated Board Position:")
    print(board)

# Main game loop
while not board.is_game_over():
    if board.turn == chess.WHITE:
        legal_moves = [board.san(move) for move in board.legal_moves]
        print("\nYour move. Legal moves:", legal_moves)
        user_move = input("Enter your move in SAN: ")
        while user_move not in legal_moves:
            print("Invalid move. Please choose from the legal moves.")
            user_move = input("Enter your move in SAN: ")
        board.push_san(user_move)
    else:
        move_count = len(board.move_stack)
        game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)
        select_best_move(board, selected_player_id, game_phase)

# End game display
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())



Your move. Legal moves: ['Nh3', 'Nf3', 'Nc3', 'Na3', 'h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3', 'h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4']
Invalid move. Please choose from the legal moves.
Stockfish Top Moves (UCI): [Move.from_uci('e7e5'), Move.from_uci('c7c5'), Move.from_uci('e7e6'), Move.from_uci('g7g6'), Move.from_uci('g8f6'), Move.from_uci('d7d5'), Move.from_uci('c7c6'), Move.from_uci('a7a6'), Move.from_uci('d7d6'), Move.from_uci('b8c6')]
AI selected move: c7c6 (based on model ranking)

Updated Board Position:
r n b q k b n r
p p . p p p p p
. . p . . . . .
. . . . . . . .
. . . . . . . .
P . . . . . . .
. P P P P P P P
R N B Q K B N R

Your move. Legal moves: ['Nh3', 'Nf3', 'Nc3', 'Ra2', 'a4', 'h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4']
Stockfish Top Moves (UCI): [Move.from_uci('e7e5'), Move.from_uci('d7d5'), Move.from_uci('g8f6'), Move.from_uci('d8c7'), Move.from_uci('e7e6'), Move.from_uci('d8a5'), Move.from_uci('g7g6'), Move.fro

In [ ]:
import chess
import pandas as pd
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load Piece Position and Time models, encoders, and Stockfish engine
piece_model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
time_model_path = r"C:\Users\shaik\Desktop\time_v2.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
scaler_X_path = r"C:\Users\shaik\Desktop\scaler_X.pkl"
scaler_y_path = r"C:\Users\shaik\Desktop\scaler_y.pkl"
stockfish_path = r"C:\Users\shaik\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

# Load models and scalers
piece_model = xgb.Booster()
piece_model.load_model(piece_model_path)
time_model = xgb.Booster()
time_model.load_model(time_model_path)
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)
with open(scaler_X_path, 'rb') as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, 'rb') as f:
    scaler_y = pickle.load(f)

# Initialize the chess board and Stockfish engine
board = chess.Board()
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Load player encoder and dataset
data_path = r"C:\Users\shaik\Desktop\updated_time_control_rest_1.csv"
df = pd.read_csv(data_path)
top_50_players = pd.concat([df['White'], df['Black']]).value_counts().head(50).index.tolist()
player_encoder = LabelEncoder()
player_encoder.fit([player.lower() for player in pd.concat([df['White'], df['Black']]).unique()])
selected_player = top_50_players[0]
selected_player_id = player_encoder.transform([selected_player.lower()])[0]

# Function to convert board to features
def board_to_features(board):
    piece_mapping = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6, 'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6, '.': 0}
    return np.array([piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES])

# Extract combined features for both models
def extract_combined_features(board, player_id, game_phase):
    board_features = board_to_features(board)
    game_mode_encoded = 1  # Example: 1 for Rapid; adjust based on actual game mode
    white_time_left_processed = 180  # Example time, adjust as per game state
    black_time_left_processed = 180
    total_seconds = 180
    piece_count = 32 - len(board.move_stack)
    time_features = np.array([game_mode_encoded, white_time_left_processed, black_time_left_processed, total_seconds, game_phase, piece_count])
    scaled_time_features = scaler_X.transform(time_features.reshape(1, -1))
    return board_features, scaled_time_features

# Function to select best move and predict time taken
def select_best_move_and_time(board, player_id, game_phase):
    # Get Stockfish's top moves
    result = engine.analyse(board, chess.engine.Limit(depth=10), multipv=10)
    stockfish_moves_uci = [info['pv'][0] for info in result if info['pv'][0] in board.legal_moves]
    
    print("Stockfish Top Moves (UCI):", stockfish_moves_uci)

    # Extract features and predict best move
    features_list = []
    for move_uci in stockfish_moves_uci:
        board.push(move_uci)
        board_features, scaled_time_features = extract_combined_features(board, player_id, game_phase)
        features_list.append(board_features)
        board.pop()
    dmatrix = xgb.DMatrix(np.array(features_list))
    predictions = piece_model.predict(dmatrix)
    best_move_index = np.argmax(predictions)
    best_move_uci = stockfish_moves_uci[best_move_index]

    # Apply best move
    board.push(best_move_uci)

    # Predict time taken for the best move
    time_dmatrix = xgb.DMatrix(scaled_time_features)
    predicted_time_scaled = time_model.predict(time_dmatrix)[0]
    predicted_time = scaler_y.inverse_transform([[predicted_time_scaled]])[0][0]
    
    # Display move and predicted time
    print(f"AI selected move: {best_move_uci} (Predicted time: {np.expm1(predicted_time):.2f} seconds)")

# Main game loop
while not board.is_game_over():
    if board.turn == chess.WHITE:
        legal_moves = [board.san(move) for move in board.legal_moves]
        print("\nYour move. Legal moves:", legal_moves)
        user_move = input("Enter your move in SAN: ")
        while user_move not in legal_moves:
            print("Invalid move. Please choose from the legal moves.")
            user_move = input("Enter your move in SAN: ")
        board.push_san(user_move)
    else:
        move_count = len(board.move_stack)
        game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)
        select_best_move_and_time(board, selected_player_id, game_phase)

# End game display
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())



Your move. Legal moves: ['Nh3', 'Nf3', 'Nc3', 'Na3', 'h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3', 'h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4']
Stockfish Top Moves (UCI): [Move.from_uci('d7d5'), Move.from_uci('e7e5'), Move.from_uci('g8f6'), Move.from_uci('c7c5'), Move.from_uci('b8c6'), Move.from_uci('c7c6'), Move.from_uci('h7h6'), Move.from_uci('a7a6'), Move.from_uci('e7e6'), Move.from_uci('g7g6')]
AI selected move: g8f6 (Predicted time: 7.08 seconds)

Your move. Legal moves: ['Ng5', 'Nf4', 'Ng1', 'Rg1', 'Nc3', 'Na3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4']
Stockfish Top Moves (UCI): [Move.from_uci('d7d5'), Move.from_uci('e7e5'), Move.from_uci('b8c6'), Move.from_uci('c7c5'), Move.from_uci('h7h6'), Move.from_uci('e7e6'), Move.from_uci('c7c6'), Move.from_uci('d7d6'), Move.from_uci('a7a6'), Move.from_uci('a7a5')]
AI selected move: e7e5 (Predicted time: 7.08 seconds)

Your move. Legal moves: ['Ng5', 'Nf4', 'Ng1', 'Rg1', 'Nc3', 'Ra2', 'a4', 

## working model which predict move as well as time


In [ ]:
import chess
import pandas as pd
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load Piece Position and Time models, encoders, and Stockfish engine
piece_model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
time_model_path = r"C:\Users\shaik\Desktop\time_v2.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
scaler_X_path = r"C:\Users\shaik\Desktop\scaler_X.pkl"
scaler_y_path = r"C:\Users\shaik\Desktop\scaler_y.pkl"
stockfish_path = r"C:\Users\shaik\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

# Load models and scalers
piece_model = xgb.Booster()
piece_model.load_model(piece_model_path)
time_model = xgb.Booster()
time_model.load_model(time_model_path)
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)
with open(scaler_X_path, 'rb') as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, 'rb') as f:
    scaler_y = pickle.load(f)

# Initialize the chess board and Stockfish engine
board = chess.Board()
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Load player encoder and dataset
data_path = r"C:\Users\shaik\Desktop\updated_time_control_rest_1.csv"
df = pd.read_csv(data_path)
top_50_players = pd.concat([df['White'], df['Black']]).value_counts().head(50).index.tolist()
player_encoder = LabelEncoder()
player_encoder.fit([player.lower() for player in pd.concat([df['White'], df['Black']]).unique()])
selected_player = top_50_players[0]
selected_player_id = player_encoder.transform([selected_player.lower()])[0]

# Function to convert board to features
def board_to_features(board):
    piece_mapping = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6, 'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6, '.': 0}
    return np.array([piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES])

# Extract combined features for both models
def extract_combined_features(board, player_id, game_phase):
    board_features = board_to_features(board)
    game_mode_encoded = 1  # Example: 1 for Rapid; adjust based on actual game mode
    white_time_left_processed = 180  # Example time, adjust as per game state
    black_time_left_processed = 180
    total_seconds = 180
    piece_count = 32 - len(board.move_stack)
    time_features = np.array([game_mode_encoded, white_time_left_processed, black_time_left_processed, total_seconds, game_phase, piece_count])
    scaled_time_features = scaler_X.transform(time_features.reshape(1, -1))
    return board_features, scaled_time_features

# Function to select best move and predict time taken
def select_best_move_and_time(board, player_id, game_phase):
    # Get Stockfish's top moves
    result = engine.analyse(board, chess.engine.Limit(depth=10), multipv=10)
    stockfish_moves_uci = [info['pv'][0] for info in result if info['pv'][0] in board.legal_moves]
    
    print("Stockfish Top Moves (UCI):", stockfish_moves_uci)

    # Extract features and predict best move
    features_list = []
    for move_uci in stockfish_moves_uci:
        board.push(move_uci)
        board_features, scaled_time_features = extract_combined_features(board, player_id, game_phase)
        features_list.append(board_features)
        board.pop()
    dmatrix = xgb.DMatrix(np.array(features_list))
    predictions = piece_model.predict(dmatrix)
    best_move_index = np.argmax(predictions)
    best_move_uci = stockfish_moves_uci[best_move_index]

    # Apply best move
    board.push(best_move_uci)

    # Predict time taken for the best move
    time_dmatrix = xgb.DMatrix(scaled_time_features)
    predicted_time_scaled = time_model.predict(time_dmatrix)[0]
    predicted_time = scaler_y.inverse_transform([[predicted_time_scaled]])[0][0]
    
    # Display move, predicted time, and updated board
    print(f"AI selected move: {best_move_uci} (Predicted time: {np.expm1(predicted_time):.2f} seconds)")
    print("\nUpdated Board Position:")
    print(board)

# Main game loop
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Display board and legal moves for user
        print("\nCurrent Board Position:")
        print(board)
        legal_moves = [board.san(move) for move in board.legal_moves]
        print("Your move. Legal moves:", legal_moves)
        
        user_move = input("Enter your move in SAN: ")
        while user_move not in legal_moves:
            print("Invalid move. Please choose from the legal moves.")
            user_move = input("Enter your move in SAN: ")
        
        board.push_san(user_move)
    else:
        move_count = len(board.move_stack)
        game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)
        select_best_move_and_time(board, selected_player_id, game_phase)

# End game display
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())



Current Board Position:
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
Your move. Legal moves: ['Nh3', 'Nf3', 'Nc3', 'Na3', 'h3', 'g3', 'f3', 'e3', 'd3', 'c3', 'b3', 'a3', 'h4', 'g4', 'f4', 'e4', 'd4', 'c4', 'b4', 'a4']
Stockfish Top Moves (UCI): [Move.from_uci('d7d5'), Move.from_uci('e7e5'), Move.from_uci('g8f6'), Move.from_uci('c7c5'), Move.from_uci('b8c6'), Move.from_uci('c7c6'), Move.from_uci('h7h6'), Move.from_uci('a7a6'), Move.from_uci('e7e6'), Move.from_uci('g7g6')]
AI selected move: g8f6 (Predicted time: 7.08 seconds)

Updated Board Position:
r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R

Current Board Position:
r n b q k b . r
p p p p p p p p
. . . . . n . .
. . . . . . . .
. . . . . . . .
. . . . . . . N
P P P P P P P P
R N B Q K B . R
Your move. Legal moves: ['Ng5', 'Nf4', 'Ng1', 'Rg1', 'Nc3', 'Na3', 'g3', 'f3', '

Updated Code with Player and Mode Selection

In [ ]:
import chess
import pandas as pd
import chess.engine
import numpy as np
import xgboost as xgb
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load Piece Position and Time models, encoders, and Stockfish engine
piece_model_path = r"C:\Users\shaik\Desktop\50_player_code.json"
time_model_path = r"C:\Users\shaik\Desktop\time_v2.json"
encoder_path = r"C:\Users\shaik\Desktop\move_encoder.pkl"
scaler_X_path = r"C:\Users\shaik\Desktop\scaler_X.pkl"
scaler_y_path = r"C:\Users\shaik\Desktop\scaler_y.pkl"
stockfish_path = r"C:\Users\shaik\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"

# Load models and scalers
piece_model = xgb.Booster()
piece_model.load_model(piece_model_path)
time_model = xgb.Booster()
time_model.load_model(time_model_path)
with open(encoder_path, 'rb') as f:
    move_encoder = pickle.load(f)
with open(scaler_X_path, 'rb') as f:
    scaler_X = pickle.load(f)
with open(scaler_y_path, 'rb') as f:
    scaler_y = pickle.load(f)

# Initialize the chess board and Stockfish engine
board = chess.Board()
engine = chess.engine.SimpleEngine.popen_uci(stockfish_path)

# Load player encoder and dataset
data_path = r"C:\Users\shaik\Desktop\updated_time_control_rest_1.csv"
df = pd.read_csv(data_path)
top_50_players = pd.concat([df['White'], df['Black']]).value_counts().head(50).index.tolist()
player_encoder = LabelEncoder()
player_encoder.fit([player.lower() for player in pd.concat([df['White'], df['Black']]).unique()])

# Prompt user to select a player
print("Top 50 Players:", top_50_players)
selected_player = input("Enter the name of the player you want to play against: ").strip().lower()
while selected_player not in [player.lower() for player in top_50_players]:
    print("Invalid player name. Please choose from the top 50 players.")
    selected_player = input("Enter the name of the player you want to play against: ").strip().lower()

selected_player_id = player_encoder.transform([selected_player])[0]

# Prompt user to select a game mode
game_mode_options = {'rapid': 1, 'classic': 2, 'blitz': 0}
print("Game modes available: Rapid, Classic, Blitz")
selected_mode = input("Select the game mode (Rapid, Classic, Blitz): ").strip().lower()
while selected_mode not in game_mode_options:
    print("Invalid mode. Please choose Rapid, Classic, or Blitz.")
    selected_mode = input("Select the game mode (Rapid, Classic, Blitz): ").strip().lower()

game_mode_encoded = game_mode_options[selected_mode]
print(f"Selected Player: {selected_player.capitalize()}, Mode: {selected_mode.capitalize()}")

# Function to convert board to features
def board_to_features(board):
    piece_mapping = {'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6, 'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6, '.': 0}
    return np.array([piece_mapping[board.piece_at(square).symbol()] if board.piece_at(square) else 0 for square in chess.SQUARES])

# Extract combined features for both models
def extract_combined_features(board, player_id, game_phase):
    board_features = board_to_features(board)
    white_time_left_processed = 180  # Example time, adjust as per game state
    black_time_left_processed = 180
    total_seconds = 180
    piece_count = 32 - len(board.move_stack)
    time_features = np.array([game_mode_encoded, white_time_left_processed, black_time_left_processed, total_seconds, game_phase, piece_count])
    scaled_time_features = scaler_X.transform(time_features.reshape(1, -1))
    return board_features, scaled_time_features

# Function to select best move and predict time taken
def select_best_move_and_time(board, player_id, game_phase):
    # Get Stockfish's top moves
    result = engine.analyse(board, chess.engine.Limit(depth=10), multipv=10)
    stockfish_moves_uci = [info['pv'][0] for info in result if info['pv'][0] in board.legal_moves]
    
    print("Stockfish Top Moves (UCI):", stockfish_moves_uci)

    # Extract features and predict best move
    features_list = []
    for move_uci in stockfish_moves_uci:
        board.push(move_uci)
        board_features, scaled_time_features = extract_combined_features(board, player_id, game_phase)
        features_list.append(board_features)
        board.pop()
    dmatrix = xgb.DMatrix(np.array(features_list))
    predictions = piece_model.predict(dmatrix)
    best_move_index = np.argmax(predictions)
    best_move_uci = stockfish_moves_uci[best_move_index]

    # Apply best move
    board.push(best_move_uci)

    # Predict time taken for the best move
    time_dmatrix = xgb.DMatrix(scaled_time_features)
    predicted_time_scaled = time_model.predict(time_dmatrix)[0]
    predicted_time = scaler_y.inverse_transform([[predicted_time_scaled]])[0][0]
    
    # Display move, predicted time, and updated board
    print(f"AI selected move: {best_move_uci} (Predicted time: {np.expm1(predicted_time):.2f} seconds)")
    print("\nUpdated Board Position:")
    print(board)

# Main game loop
while not board.is_game_over():
    if board.turn == chess.WHITE:
        # Display board and legal moves for user
        print("\nCurrent Board Position:")
        print(board)
        legal_moves = [board.san(move) for move in board.legal_moves]
        print("Your move. Legal moves:", legal_moves)
        
        user_move = input("Enter your move in SAN: ")
        while user_move not in legal_moves:
            print("Invalid move. Please choose from the legal moves.")
            user_move = input("Enter your move in SAN: ")
        
        board.push_san(user_move)
    else:
        move_count = len(board.move_stack)
        game_phase = 1 if move_count <= 15 else (2 if move_count <= 40 else 3)
        select_best_move_and_time(board, selected_player_id, game_phase)

# End game display
print("Game Over!")
print("Final Board Position:")
print(board)
print("Result:", board.result())


Top 50 Players: ['sarana, alexey', 'firouzja, alireza', 'nakamura, hikaru', 'caruana, fabiano', 'duda, jan-krzysztof', 'carlsen, magnus', 'vachier-lagrave, maxime', 'aronian, levon', 'niemann, hans moke', 'nepomniachtchi, ian', 'giri, anish', 'abdusattorov, nodirbek', 'keymer, vincent', 'so, wesley', 'vidit, santosh gujrathi', 'maghsoodloo, parham', 'mamedyarov, shakhriyar', '?', 'donchenko, alexander', 'tabatabaei, m. amin', 'le, quang liem', 'dominguez perez, leinier', 'artemiev, vladislav', 'rapport, richard', 'erigaisi arjun', 'yu, yangyi', 'gukesh d', 'praggnanandhaa r', 'praggnanandhaa, r', 'erigaisi, arjun', 'ding, liren', 'gukesh, d', 'wei, yi', 'grischuk, alexander', 'andreikin, dmitry', 'xiong, jeffery', 'martinez alcantara, jose eduardo', 'paravyan, david', 'dubov, daniil', 'lazavik, denis', 'bortnyk, olexandr', 'fedoseev, vladimir', 'nihal, sarin', 'nihal sarin', 'sevian, samuel', 'kamsky, gata', 'sadhwani, raunak', 'deac, bogdan-daniel', 'yoo, christopher woojin', 'wang, h